In [1]:
import pandas as pd 
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import Dense

In [3]:
df = pd.read_csv('./chicken.csv')

df.head()

,index,value
0,2001-08,65.58
1,2001-09,66.48
2,2001-10,65.70
3,2001-11,64.33
4,2001-12,63.23


In [4]:
df.dtypes

index     object
value    float64
dtype: object

In [5]:
df['index'] = df['index'].astype('datetime64[ns]')

df.head()

,index,value
0,2001-08-01,65.58
1,2001-09-01,66.48
2,2001-10-01,65.70
3,2001-11-01,64.33
4,2001-12-01,63.23


In [6]:
df = df.set_index('index')

df.head()

,value
index,
2001-08-01,65.58
2001-09-01,66.48
2001-10-01,65.70
2001-11-01,64.33
2001-12-01,63.23


In [7]:
X1 = df.shift(1)

X1.head()

,value
index,
2001-08-01,NaN
2001-09-01,65.58
2001-10-01,66.48
2001-11-01,65.70
2001-12-01,64.33


In [8]:
X2 = df.shift(2)

X2.head()

,value
index,
2001-08-01,NaN
2001-09-01,NaN
2001-10-01,65.58
2001-11-01,66.48
2001-12-01,65.70


In [9]:
X3 = df.shift(3)

X3.head()

,value
index,
2001-08-01,NaN
2001-09-01,NaN
2001-10-01,NaN
2001-11-01,65.58
2001-12-01,66.48


In [10]:
X4 = df.shift(4)

X4.head()

,value
index,
2001-08-01,NaN
2001-09-01,NaN
2001-10-01,NaN
2001-11-01,NaN
2001-12-01,65.58


In [11]:
df_time = X1.copy()

df_time

,value
index,
2001-08-01,NaN
2001-09-01,65.58
2001-10-01,66.48
2001-11-01,65.70
2001-12-01,64.33
...,...
2016-03-01,112.10
2016-04-01,111.56
2016-05-01,111.55


In [12]:
df_time = df_time.rename(columns={'value': 'L1'})

df_time.head()

,L1
index,
2001-08-01,NaN
2001-09-01,65.58
2001-10-01,66.48
2001-11-01,65.70
2001-12-01,64.33


In [13]:
df_time['L2'] = X2['value']
df_time['L3'] = X3['value']
df_time['L4'] = X4['value']
df_time['Data'] = df['value']

df_time.head()

,L1,L2,L3,L4,Data
index,,,,,
2001-08-01,NaN,NaN,NaN,NaN,65.58
2001-09-01,65.58,NaN,NaN,NaN,66.48
2001-10-01,66.48,65.58,NaN,NaN,65.70
2001-11-01,65.70,66.48,65.58,NaN,64.33
2001-12-01,64.33,65.70,66.48,65.58,63.23


In [14]:
df_time = df_time.iloc[4:,]

df_time.head()

,L1,L2,L3,L4,Data
index,,,,,
2001-12-01,64.33,65.70,66.48,65.58,63.23
2002-01-01,63.23,64.33,65.70,66.48,62.94
2002-02-01,62.94,63.23,64.33,65.70,62.92
2002-03-01,62.92,62.94,63.23,64.33,62.73
2002-04-01,62.73,62.92,62.94,63.23,62.50


In [15]:
df_time = np.round(np.log(df_time), 2)

df_time

,L1,L2,L3,L4,Data
index,,,,,
2001-12-01,4.16,4.19,4.20,4.18,4.15
2002-01-01,4.15,4.16,4.19,4.20,4.14
2002-02-01,4.14,4.15,4.16,4.19,4.14
2002-03-01,4.14,4.14,4.15,4.16,4.14
2002-04-01,4.14,4.14,4.14,4.15,4.14
...,...,...,...,...,...
2016-03-01,4.72,4.72,4.73,4.73,4.71
2016-04-01,4.71,4.72,4.72,4.73,4.71
2016-05-01,4.71,4.71,4.72,4.72,4.72


In [16]:
minimal = np.min(df_time)
maximal = np.max(df_time)

print("minimal", minimal)
print("maximal", minimal)

minimal 4.12
maximal 4.12


In [17]:
def range_data(unit, minimal, maximal):
    return (unit - minimal) / (maximal - minimal)

In [18]:
df_time = df_time.apply(lambda x: range_data(x, minimal=minimal, maximal=maximal))

df_time.head()

,L1,L2,L3,L4,Data
index,,,,,
2001-12-01,0.063492,0.111111,0.126984,0.095238,0.047619
2002-01-01,0.047619,0.063492,0.111111,0.126984,0.031746
2002-02-01,0.031746,0.047619,0.063492,0.111111,0.031746
2002-03-01,0.031746,0.031746,0.047619,0.063492,0.031746
2002-04-01,0.031746,0.031746,0.031746,0.047619,0.031746


In [19]:
y = df_time['Data']

y_train = y.iloc[:170]
y_test = y.iloc[171:]

In [20]:
y_train.head()

index
2001-12-01    0.047619
2002-01-01    0.031746
2002-02-01    0.031746
2002-03-01    0.031746
2002-04-01    0.031746
Name: Data, dtype: float64

In [21]:
y_test.head()

index
2016-03-01    0.936508
2016-04-01    0.936508
2016-05-01    0.952381
2016-06-01    0.952381
2016-07-01    0.936508
Name: Data, dtype: float64

In [22]:
X_train = df_time.iloc[:170,:4]
X_train = X_train.reset_index()[['L1', 'L2', 'L3', 'L4']]

X_test = df_time.iloc[171:,4]

In [23]:
X_train.head()

,L1,L2,L3,L4
0,0.063492,0.111111,0.126984,0.095238
1,0.047619,0.063492,0.111111,0.126984
2,0.031746,0.047619,0.063492,0.111111
3,0.031746,0.031746,0.047619,0.063492
4,0.031746,0.031746,0.031746,0.047619


In [24]:
X_test.head()

index
2016-03-01    0.936508
2016-04-01    0.936508
2016-05-01    0.952381
2016-06-01    0.952381
2016-07-01    0.936508
Name: Data, dtype: float64

In [25]:
model = Sequential()

model.add(SimpleRNN(activation='tanh', units=2))
model.add(Dense(units=1))

model.compile()

In [32]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50


ValueError: in user code:

    File "c:\Users\KG\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\KG\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\KG\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\KG\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\KG\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Membuat data deret waktu sintetis
n = 100
time = np.arange(0, n, 1)
sin_wave = np.sin(0.1 * time) + 0.1 * np.random.randn(n)

# Membagi data menjadi data latih dan data validasi
train_data = sin_wave[:80]
val_data = sin_wave[80:]

# Fungsi untuk membuat dataset dengan timesteps
def create_dataset(data, timesteps):
    X, y = [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i+timesteps])
        y.append(data[i+timesteps])
    return np.array(X), np.array(y)

timesteps = 10
X_train, y_train = create_dataset(train_data, timesteps)
X_val, y_val = create_dataset(val_data, timesteps)

# print

# # Membangun model RNN sederhana dengan SimpleRNN
# model = keras.Sequential([
#     layers.SimpleRNN(32, input_shape=(timesteps, 1)),
#     layers.Dense(1)
# ])

# # Kompilasi model
# model.compile(optimizer='adam', loss='mean_squared_error')

# # Melatih model
# model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

# # Membuat prediksi
# X_test = np.array([val_data[i:i+timesteps] for i in range(len(val_data) - timesteps)])
# y_pred = model.predict(X_test)

# # Menampilkan hasil prediksi
# import matplotlib.pyplot as plt
# plt.plot(val_data[timesteps:], label="Actual")
# plt.plot(y_pred, label="Predicted")
# plt.legend()
# plt.show()

In [28]:
print(X_train)

[[ 0.03226425  0.18489705  0.31140936  0.34750826  0.14133218  0.52275222
   0.54922993  0.67700068  0.76180579  0.80141562]
 [ 0.18489705  0.31140936  0.34750826  0.14133218  0.52275222  0.54922993
   0.67700068  0.76180579  0.80141562  0.89270768]
 [ 0.31140936  0.34750826  0.14133218  0.52275222  0.54922993  0.67700068
   0.76180579  0.80141562  0.89270768  0.7546712 ]
 [ 0.34750826  0.14133218  0.52275222  0.54922993  0.67700068  0.76180579
   0.80141562  0.89270768  0.7546712   0.93070685]
 [ 0.14133218  0.52275222  0.54922993  0.67700068  0.76180579  0.80141562
   0.89270768  0.7546712   0.93070685  1.10071355]
 [ 0.52275222  0.54922993  0.67700068  0.76180579  0.80141562  0.89270768
   0.7546712   0.93070685  1.10071355  0.83997427]
 [ 0.54922993  0.67700068  0.76180579  0.80141562  0.89270768  0.7546712
   0.93070685  1.10071355  0.83997427  0.99085466]
 [ 0.67700068  0.76180579  0.80141562  0.89270768  0.7546712   0.93070685
   1.10071355  0.83997427  0.99085466  1.08706509]
 

In [29]:
print(X_test)

index
2016-03-01    0.936508
2016-04-01    0.936508
2016-05-01    0.952381
2016-06-01    0.952381
2016-07-01    0.936508
Name: Data, dtype: float64


In [30]:
print(y_train)

[ 0.89270768  0.7546712   0.93070685  1.10071355  0.83997427  0.99085466
  1.08706509  1.08684729  0.9843973   0.86960018  0.83516955  0.91809066
  0.85923023  0.81066938  0.54889436  0.71696993  0.56872123  0.38940551
  0.37318431  0.12180731 -0.06404863  0.0290137  -0.1845125  -0.1280674
 -0.22319913 -0.33679587 -0.47726262 -0.59749558 -0.64325846 -0.70926783
 -0.80560782 -0.96058046 -0.89631692 -0.91105211 -0.79614802 -0.98217651
 -1.16622906 -0.94434847 -0.97048681 -1.20865289 -0.88906225 -0.85073676
 -0.79465233 -0.95925454 -0.6882646  -0.78490809 -0.73913319 -0.67217198
 -0.42773646 -0.453194   -0.23093265 -0.26529233 -0.04303083 -0.05288758
  0.16934254  0.49644748  0.18821665  0.41756542  0.39553284  0.60665537
  0.69968685  0.73698437  0.83402156  0.76096974  1.00507762  0.98269922
  1.19507298  1.14679044  1.06509758  1.21698894]


In [31]:
print(y_test)

index
2016-03-01    0.936508
2016-04-01    0.936508
2016-05-01    0.952381
2016-06-01    0.952381
2016-07-01    0.936508
Name: Data, dtype: float64
